In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,roc_curve,auc, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import precision_score, recall_score, precision_recall_fscore_support
from sklearn.metrics import f1_score,precision_recall_curve
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
credit_data= pd.read_csv("/Users/ganeshjadhav/Desktop/ML_Projects/LiveClass/Notebooks2/CreditCardData.csv")

In [4]:
credit_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
1,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
2,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
3,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
4,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1


In [3]:
credit_data.shape

(10492, 31)

In [5]:
X= credit_data.drop('Class', axis= 1)
y= credit_data['Class']

In [6]:
#Feature EngineeringPowerTransformer
from sklearn.preprocessing import PowerTransformer
power = PowerTransformer(method='yeo-johnson', standardize=True)
df = power.fit_transform(X)

In [9]:
pd.DataFrame(df)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-2.412968,-1.098091,0.899669,-0.853623,1.828163,-0.294664,-1.029474,-1.209057,0.825528,-1.884694,...,0.135883,0.476663,-0.072812,-0.717845,0.492346,0.058854,0.449719,0.509067,-0.435869,-1.934999
1,-2.408147,-1.285209,-1.702840,0.722170,1.177573,0.853147,-0.714194,0.210144,-0.103058,-0.179657,...,2.682045,0.607316,0.551213,2.114744,-0.537478,0.507892,-0.230038,-0.568440,0.102236,1.781138
2,-2.206409,-1.095674,0.803819,-0.338749,1.194679,-0.452218,0.081214,0.382627,-0.293461,-0.155193,...,-0.578592,-0.290255,-1.205767,0.259359,-0.213109,-0.311609,-1.199464,0.033041,-0.465388,1.366695
3,-2.106188,-1.583982,0.604075,-1.159258,1.335503,-0.608697,-1.280729,-1.561190,-0.207996,-0.162321,...,-0.247317,0.527786,0.205651,-0.673522,-0.157999,0.454860,-1.504713,-1.681716,2.469461,0.601185
4,-2.086232,0.675519,1.428808,-1.588434,2.082262,2.487024,-0.968668,1.357609,-0.347980,-0.901690,...,-0.015470,-0.374208,-0.923202,-1.010422,-2.321570,3.000477,1.157730,-0.071090,0.432211,-1.474016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,0.971580,-0.595641,0.872454,1.618755,1.896304,0.055139,1.037124,-0.135495,0.433981,-1.350997,...,0.349773,-0.215144,-0.842067,-0.153337,0.673197,-0.021748,0.482237,0.455009,0.312784,-1.558014
10488,1.130041,-0.394485,0.464991,-0.408239,-0.345834,0.854739,-0.043752,0.780125,-0.096067,0.045526,...,0.202601,-0.012917,0.352860,-0.469015,-0.111485,-0.093144,1.194766,0.746589,0.784790,0.140366
10489,1.320575,1.579636,-0.523659,-0.862573,-0.824658,-0.047469,-0.176529,-0.181182,-0.193461,-0.547899,...,0.182677,-0.037801,-0.366058,0.134114,-1.656662,-0.205949,-0.804814,-0.146908,-0.201816,0.719891
10490,-0.894436,0.580196,-0.018384,0.089698,0.395325,-0.206876,-0.183649,-0.106954,0.021621,-0.007147,...,-0.239038,-0.199639,-0.897070,0.263033,0.183323,0.041773,0.289038,-0.095300,0.051810,-0.133864


In [6]:
df= pd.DataFrame(data= df, columns= X.columns)
from imblearn.over_sampling import SMOTE
from collections import Counter
smt = SMOTE(random_state=2425, n_jobs=-1,sampling_strategy='auto', k_neighbors=5)
X_sm, y_sm = smt.fit_resample(df, y)
print('Resampled dataset shape {}'.format(Counter(y_sm)))
print('Before Resampled dataset shape {}'.format(Counter(y)))

Resampled dataset shape Counter({1: 10000, 0: 10000})
Before Resampled dataset shape Counter({0: 10000, 1: 492})


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size = 0.30, random_state = 440)

After using Hyperparameter tunning we found best parameter for Random forest and using on it

In [8]:
RF_model=Pipeline([('scalar3',StandardScaler()),
                     ('pca3',PCA()),
                     ('rf_classifier',RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 100, max_depth=8, criterion='gini', min_samples_leaf= 15))])

In [9]:
RF_model.fit(x_train, y_train)

Pipeline(steps=[('scalar3', StandardScaler()), ('pca3', PCA()),
                ('rf_classifier',
                 RandomForestClassifier(max_depth=8, max_features='auto',
                                        min_samples_leaf=15,
                                        random_state=42))])

In [10]:
pred=RF_model.predict(x_test)

In [11]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

Accuracy for Random Forest on CV data:  0.9423333333333334


In [12]:
# Save the Modle to file in the current working directory
import pickle
Pkl_Filename = "Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(RF_model, file)
print("RF_model Saved")

RF_model Saved


# Now we are testing the data

In [15]:
# Load the Model back from file
Pkl_Filename = "Model.pkl"  
with open(Pkl_Filename, 'rb') as file:  
  Pickled_RF_model= pickle.load(file)

Pickled_RF_model

Pipeline(steps=[('scalar3', StandardScaler()), ('pca3', PCA()),
                ('rf_classifier',
                 RandomForestClassifier(max_depth=8, max_features='auto',
                                        min_samples_leaf=15,
                                        random_state=42))])

In [16]:
# Use the Reloaded Model to 
# Calculate the accuracy score and predict target values

# Calculate the Score 
score = Pickled_RF_model.score(x_test, y_test)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score))  

# Predict the Labels using the reloaded Model
Ypredict = Pickled_RF_model.predict(x_test)  

Ypredict

Test score: 94.23 %


array([0, 1, 1, ..., 1, 1, 0])

In [17]:
#predict the data using only one row

features= np.array([-0.39953817, -0.97712131,  1.24544234,  0.94905287,  1.7934207 ,
       -0.72105241,  0.71068923, -0.77640272,  1.36614126, -1.42028025,
        0.790728  ,  0.89664203, -0.02611889, -0.61222435,  1.12123451,
        1.2181234 ,  0.33202657,  0.30275497,  0.838198  ,  0.55880226,
        0.35222467,  0.51204671,  1.14737082, -0.18496812, -0.09257469,
       -0.5097014 ,  0.79919997,  0.73473953,  0.42037289, -0.29209222])

In [18]:
Ypredict = Pickled_RF_model.predict([features])  

Ypredict

array([0])